In [48]:
#imports
import math
import io

In [58]:
#data pre-processing
data = []
with io.open('banknote_data.txt', encoding='latin-1') as myfile:
    for i in myfile.readlines():
        data.append(i)
dataa= []
for i in data:
    i = i.split(",")
    for j in range(len(i)):
        t = float(i[j])
        i[j] = t
    dataa.append(i)
data = dataa
train = data[0:1234]
test = data[1234:-1]

In [59]:
#split 
def split(data,column,value):
    l,r = [],[]
    for row in data:
        if row[column] >= value:
            l.append(row)
        else:
            r.append(row)
    return l,r

#entropy of split
def entropy_split(groups,classes):
    ent = 0
    for group in groups:
        normalized_group_size = len(group)/len(data)
        group_sum = 0
        for i in classes:
            if len(group)==0:
                continue
            p = [row[-1] for row in group].count(i)/len(group)
            if p == 0:
                continue
            group_sum -= p*math.log2(p)
        ent += normalized_group_size * group_sum
    return ent

#best_split
def best_split(rows):
    entropy_dataset = 0
    class_values = list(set(row[-1] for row in data))
    for i in class_values:
        p = [row[-1] for row in data].count(i) / len(data)
        if p == 0:
            continue
        entropy_dataset -= p*math.log2(p)
    data_entropy,best_info_gain,chosen_column,chosen_value = entropy_dataset,0,0,0
    for column in range(0,len(rows[0])-1):
        values = set([row[column] for row in rows])
        for value in values:
            class_values = list(set(row[-1] for row in data))
            left,right = split(rows,column,value)
            if len(left)==0 or len(right)==0:
                continue
            entropy_of_split = entropy_split([left,right],class_values)
            curr_info_gain = data_entropy - entropy_of_split
            if curr_info_gain > best_info_gain:
                best_info_gain = curr_info_gain
                chosen_column = column
                chosen_value = value
    return chosen_column,chosen_value,best_info_gain

In [60]:
class Leaf:
    def __init__(self,rows):
        classes = list(set(row[-1] for row in rows))
        if len(classes)>1:
            p = [row[-1] for row in rows].count(classes[0]) / len(rows)
            n = [row[-1] for row in rows].count(classes[1]) / len(rows)
            if p>n:
                self.prediction = classes[0]
            else:
                self.prediction = classes[1]
        else:
            self.prediction = classes[0]
        
class Branch:
    def __init__(self,left,right,ix,val):
        self.ix = ix
        self.val = val
        self.left = left
        self.right = right
    def match(self,data):
        return data[self.ix]>self.val
    
def build_tree(rows,ctr,max_depth):
    col,val,bif = best_split(rows)
    if ctr>max_depth:
        return Leaf(rows)
    elif bif == 0:
        return Leaf(rows)
    true,false = split(rows,col,val)
    ctr = ctr + 1
    left_twig = build_tree(true,ctr,max_depth)
    right_twig = build_tree(false,ctr,max_depth)
    return Branch(left_twig,right_twig,col,val)

In [61]:
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.prediction)
        return
    print (spacing + 'Is Column %s value >= %s'%(node.ix,node.val))
    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.left, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.right, spacing + "  ")

In [62]:
dt = build_tree(train,0,4)
print_tree(dt)

Is Column 0 value >= 0.3223
--> True:
  Is Column 0 value >= 1.7939
  --> True:
    Is Column 2 value >= -4.9417
    --> True:
      Is Column 0 value >= 2.0421
      --> True:
        Is Column 0 value >= 2.0922
        --> True:
          Predict 0.0
        --> False:
          Predict 0.0
      --> False:
        Is Column 0 value >= 2.0177
        --> True:
          Predict 1.0
        --> False:
          Predict 0.0
    --> False:
      Is Column 0 value >= 2.3917
      --> True:
        Predict 1.0
      --> False:
        Predict 1.0
  --> False:
    Is Column 2 value >= -2.2718
    --> True:
      Is Column 3 value >= 0.097399
      --> True:
        Is Column 2 value >= 2.0013
        --> True:
          Predict 0.0
        --> False:
          Predict 1.0
      --> False:
        Is Column 0 value >= 0.32924
        --> True:
          Predict 0.0
        --> False:
          Predict 0.0
    --> False:
      Is Column 1 value >= 7.6377
      --> True:
        Is Column 0 v

In [63]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.prediction
    if node.match(row):
        return classify(row, node.left)
    else:
        return classify(row, node.right)

In [64]:
def accuracy(data,dt):
    x = 0
    for row in data:
        if(row[-1]==classify(row, dt)):
            x += 1
    acc = (x/len(data))*100
    print("Accuracy - ", acc)
    return acc
accuracy(test,dt)

Accuracy -  97.8102189781022


97.8102189781022

In [65]:
from random import randrange

def decision_tree(train,test,max_length=2,ctr=0):
    dt = build_tree(train,ctr,max_length)
    return accuracy(test,dt)

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    testing_data= dataset[0:130]
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
        acc = algorithm(train_set, testing_data)
        scores.append(acc)
    mean_score = sum(scores)/len(scores)
    return {"scores":scores,"mean":mean_score}


In [67]:
evaluate_algorithm(data,decision_tree,9)

Accuracy -  93.84615384615384
Accuracy -  91.53846153846153
Accuracy -  91.53846153846153
Accuracy -  93.84615384615384
Accuracy -  91.53846153846153
Accuracy -  91.53846153846153
Accuracy -  94.61538461538461
Accuracy -  93.07692307692308
Accuracy -  90.76923076923077


{'scores': [93.84615384615384,
  91.53846153846153,
  91.53846153846153,
  93.84615384615384,
  91.53846153846153,
  91.53846153846153,
  94.61538461538461,
  93.07692307692308,
  90.76923076923077],
 'mean': 92.47863247863249}

In [72]:
extracted = []
testdata = data[0:130] 
for i in testdata:
    extracted.append(classify(i,dt))
extracted
#the first 150 or so rows have a class of 0 only.

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]